In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.utils import checkLogs
from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
def which_century(year):
    yield int(year / 100) + 1

def which_era(cet):
    if 1 <= cet < 4:
        era = "Targerian Regime"
    elif 4 <= cet < 8:
        era =  "Lanister Regime"
    elif 8 <= cet < 12:
        era =  "Stark Regime"
    elif 12 <= cet < 16:
        era =  "Barathion Regime"
    elif cet >= 16:
        era = "Long Winter"
    yield era

test_session(
    ie_funcs=[
        ['which_century', which_century, [int], [int]],
        ['which_era', which_era, [int], [str]]
    ],
    queries=[
    """
    new event(str, int)
    event("First Dragon", 250)
    event("Mad king", 390)
    event("Winter came", 1750)
    event("Hodor", 999)
    event("Joffery died", 799)
    
    new important_year(int)
    important_year(999)
    important_year(1750)
    important_year(250)
    
    
    important_events(EVE, Y) <- event(EVE, Y), important_year(Y)
    
    important_events_per_cet(EVE, CET) <- important_events(EVE, Y), which_century(Y) -> (CET)
    ?important_events_per_cet(EVE, CET)
    """,
    """
    important_events_per_era(EVE, ERA) <- important_events_per_cet(EVE, CET), which_era(CET) -> (ERA)
    ?important_events_per_era(EVE, ERA)
    """
    ],
    expected_outputs = [
        DataFrame({
            'EVE': ['Hodor', 'Winter came', 'First Dragon'],
            'CET': [10, 18, 3]
        }),
        DataFrame({
            'EVE': ['Hodor', 'Winter came', 'First Dragon'],
            'ERA': ['Stark Regime', 'Long Winter', 'Targerian Regime']
        })
        
    ]
)

'?important_events_per_cet(EVE,CET)'

,EVE,CET
0,First Dragon,3
1,Winter came,18
2,Hodor,10


'?important_events_per_era(EVE,ERA)'

,EVE,ERA
0,First Dragon,Targerian Regime
1,Winter came,Long Winter
2,Hodor,Stark Regime


In [ ]:
def get_population_growth_rate(year):
    if year <= 1700:
        yield 1
    elif 1700 < year <= 1900:
        yield 3
    elif 1900 < year <= 2000:
        yield 5
    else:  
        yield 1

def historical_event(year):
    if year == 1600:
        yield "Discovery of New Land"
    elif year == 1750:
        yield "Industrial Revolution"
    elif year == 1920:
        yield "Roaring Twenties"
    elif year == 2000:
        yield "Y2K Bug Panic"
    else:
        yield "No significant event"

test_session(
    ie_funcs=[
        ['get_population_growth_rate', get_population_growth_rate, [int], [int]],
        ['historical_event', historical_event, [int], [str]]
    ],
    queries=[
    """
    new year_event(int, str)
    year_event(1600, "Discovery of New Land")
    year_event(1750, "Industrial Revolution")
    year_event(1920, "Roaring Twenties")
    year_event(2000, "Y2K Bug Panic")
    
    new population_year(int)
    population_year(1600)
    population_year(1700)
    population_year(1800)
    population_year(1900)
    population_year(2000)
    population_year(1750)
    population_year(1920)
    
    population_growth_rate(Y,GROWTH) <- population_year(Y) ,get_population_growth_rate(Y) -> (GROWTH)
    
    historical_event(Y, EVENT) <- year_event(Y, EVENT)
    
    important_years(Y, EVENT, GROWTH) <- population_growth_rate(Y, GROWTH), historical_event(Y, EVENT)
    
    ?important_years(Y, EVENT, GROWTH)
    """],
    expected_outputs = [
     pd.DataFrame([
        [1600, "Discovery of New Land", 1],
        [1750, "Industrial Revolution", 3],
        [1920, "Roaring Twenties", 5],
        [2000, "Y2K Bug Panic", 5]
     ], columns=['Y', 'EVENT', 'GROWTH'])   
    ]
)

'?important_years(Y,EVENT,GROWTH)'

,Y,EVENT,GROWTH
0,1600,Discovery of New Land,1
1,2000,Y2K Bug Panic,5
2,1750,Industrial Revolution,3
3,1920,Roaring Twenties,5


In [ ]:
def technological_advancement(year):
        if year <= 1700:
            yield "Low"
        elif 1700 < year <= 1900:
            yield "Moderate"
        elif 1900 < year <= 2000:
            yield "High"
        else:
            yield "Low"

test_session(
    """
    new year_event(int, str)
    year_event(1600, "Discovery of New Land")
    year_event(1750, "Industrial Revolution")
    year_event(1920, "Roaring Twenties")
    year_event(2000, "Y2K Bug Panic")
    
    new population_year(int)
    population_year(1600)
    population_year(1700)
    population_year(1800)
    population_year(1900)
    population_year(2000)
    population_year(1750)
    population_year(1920)
    
    population_growth_rate(Y, GROWTH) <- population_year(Y), get_population_growth_rate(Y) -> (GROWTH)
    
    technological_advancement(Y, TECH) <- year_event(Y, Z), technological_advancement(Y) -> (TECH)
    
    important_years(Y, EVENT, GROWTH, TECH) <- population_growth_rate(Y, GROWTH), year_event(Y, EVENT), technological_advancement(Y, TECH)
    
    ?important_years(Y, EVENT, GROWTH, TECH)
    """,

    pd.DataFrame([
        [1600, "Discovery of New Land", 1, "Low"],
        [1750, "Industrial Revolution", 3, "Moderate"],
        [1920, "Roaring Twenties", 5, "High"],
        [2000, "Y2K Bug Panic", 5, "High"]
    ], columns=['Y', 'EVENT', 'GROWTH', 'TECH']),
    ie_funcs=[
        ['get_population_growth_rate', get_population_growth_rate, [int], [int]],
        ['technological_advancement', technological_advancement, [int], [str]]
    ]
)

'?important_years(Y,EVENT,GROWTH,TECH)'

,Y,EVENT,GROWTH,TECH
0,1600,Discovery of New Land,1,Low
1,2000,Y2K Bug Panic,5,High
2,1750,Industrial Revolution,3,Moderate
3,1920,Roaring Twenties,5,High


In [ ]:
def get_match_result(home_team, away_team, home_goals, away_goals):
    if home_goals > away_goals:
        yield "Home Win"
    elif home_goals < away_goals:
        yield "Away Win"
    else:
        yield "Draw"

def get_player_position(player_name):
    if player_name in ["Lionel Messi", "Cristiano Ronaldo"]:
        yield "Forward"
    elif player_name in ["Virgil van Dijk", "Sergio Ramos"]:
        yield "Defender"
    else:
        yield "Unknown"

test_session(
    [
    """
    new team(str)
team("Barcelona")
    team("Real Madrid")
    team("Liverpool")
    team("Juventus")
    
    new player(str, str)
    player("Lionel Messi", "Barcelona")
    player("Cristiano Ronaldo", "Juventus")
    player("Virgil van Dijk", "Liverpool")
    player("Sergio Ramos", "Real Madrid")
    
    new match(str, str, int, int)
    match("Barcelona", "Real Madrid", 3, 2)
    match("Real Madrid", "Barcelona", 1, 1)
    match("Liverpool", "Juventus", 2, 0)
    match("Juventus", "Liverpool", 0, 1)
    
    match_result(HT, AT, HG, AG, RESULT) <- match(HT, AT, HG, AG), get_match_result(HT, AT, HG, AG) -> (RESULT)
    
    player_position(P, POS) <- player(P, X), get_player_position(P) -> (POS)
    
    ?match_result(HT, AT, HG, AG, RESULT)
    
    """,
    """?player_position(P, POS) """    
    ],
    [
        pd.DataFrame([
            ['Barcelona', 'Real Madrid', 3, 2, 'Home Win'],
            ['Real Madrid', 'Barcelona', 1, 1, 'Draw'],
            ['Liverpool', 'Juventus', 2, 0, 'Home Win'],
            ['Juventus', 'Liverpool', 0, 1, 'Away Win']
        ], columns=['HT', 'AT', 'HG', 'AG', 'RESULT']),
        pd.DataFrame([
            ['Lionel Messi', 'Forward'],
            ['Cristiano Ronaldo', 'Forward'],
            ['Virgil van Dijk', 'Defender'],
            ['Sergio Ramos', 'Defender']
        ], columns=['P', 'POS'])
    ],
    ie_funcs=[
        ['get_match_result', get_match_result, [str, str, int, int], [str]],
        ['get_player_position', get_player_position, [str], [str]]
    ]

)

'?match_result(HT,AT,HG,AG,RESULT)'

,HT,AT,HG,AG,RESULT
0,Barcelona,Real Madrid,3,2,Home Win
1,Real Madrid,Barcelona,1,1,Draw
2,Liverpool,Juventus,2,0,Home Win
3,Juventus,Liverpool,0,1,Away Win


'?player_position(P,POS)'

,P,POS
0,Lionel Messi,Forward
1,Cristiano Ronaldo,Forward
2,Virgil van Dijk,Defender
3,Sergio Ramos,Defender


In [ ]:
def calc_economic_index(interest, inflation):
    yield interest + inflation

def classify_economic_index(index):
    if index > 15:
        yield "Bad"
    else:
        yield "Good"

test_session(
    """
    new economy_data(int, int, int)  # year, interest, inflation
    economy_data(2000, 4, 3)
    economy_data(2008, 8, 10)
    economy_data(2019, 3, 2)

    complex_economic_status(YEAR, STATUS) <- economy_data(YEAR, INTEREST, INFLATION), calc_economic_index(INTEREST, INFLATION) -> (INDEX), classify_economic_index(INDEX) -> (STATUS)

    ?complex_economic_status(YEAR, STATUS)
    """,
    pd.DataFrame([
        [2000, 'Good'],
        [2008, 'Bad'],
        [2019, 'Good']
    ], columns=['YEAR', 'STATUS']),
    ie_funcs=[
        ['calc_economic_index', calc_economic_index, [int, int], [int]],
        ['classify_economic_index', classify_economic_index, [int], [str]]
    ]
)


'?complex_economic_status(YEAR,STATUS)'

,YEAR,STATUS
0,2000,Good
1,2008,Bad
2,2019,Good


In [ ]:
def calc_travel_time(distance):
    yield int(distance / 1000)

def analyze_elements(elements):
    score = 0
    if 'Carbon' in elements:
        score += 5
    if 'Water' in elements:
        score += 5
    yield score


def calculate_life_probability(age, element_score, travel_time):
    yield int((element_score * 2) - (age / 1000) - travel_time)


test_session(
    """
    new exoplanet(str, int, int, str)  # name, distance, age, elements
    exoplanet("PlanetA", 4000, 10000, "Carbon,Water")
    exoplanet("PlanetB", 9000, 5000, "Hydrogen,Silicon")

    travel_time(NAME, TIME) <- exoplanet(NAME, DIST, AGE, ELEM), calc_travel_time(DIST) -> (TIME)
    element_score(NAME, SCORE) <- exoplanet(NAME, DIST, AGE, ELEM), analyze_elements(ELEM) -> (SCORE)
    life_probability(NAME, PROB) <- exoplanet(NAME, DIST, AGE, ELEM), travel_time(NAME, TIME), element_score(NAME, SCORE), calculate_life_probability(AGE, SCORE, TIME) -> (PROB)
    ?life_probability(NAME, PROB)
    """,
    pd.DataFrame([
        ['PlanetA', 6],
        ['PlanetB', -14]
    ], columns=['NAME', 'PROB']),
    ie_funcs=[
        ['calc_travel_time', calc_travel_time, [int], [int]],
        ['analyze_elements', analyze_elements, [str], [int]],
        ['calculate_life_probability', calculate_life_probability, [int, int, int], [int]]
    ]
)


'?life_probability(NAME,PROB)'

,NAME,PROB
0,PlanetA,6
1,PlanetB,-14


In [ ]:
def ie_func1(age):
    yield age + 1

def ie_func2(course, increased_age):
    if increased_age > 26:
        yield f"Senior-{course}"
    else:
        yield f"Junior-{course}"

test_session(
    """
    new student(str, int)
    student("Alice", 25)
    student("Bob", 22)
    student("Charlie", 28)
    
    new course(str, str)
    course("CS101", "Computer Science")
    course("EN101", "English")
    course("MA101", "Mathematics")
    
    new enrolled(str, str)
    enrolled("Alice", "CS101")
    enrolled("Alice", "EN101")
    enrolled("Bob", "MA101")
    enrolled("Charlie", "EN101")
    
    sample_rule3(S, A, G) <- student(S, A), enrolled(S, C), course(C, T), ie_func1(A) -> (IA), ie_func2(T, IA) -> (G)
                            
    ?sample_rule3(S, A, G)
    """,
    pd.DataFrame({
        'S': ['Alice', 'Alice', 'Bob', 'Charlie'],
        'A': [25, 25, 22, 28],
        'G': ['Junior-Computer Science', 'Junior-English', 'Junior-Mathematics', 'Senior-English']
    }),
    ie_funcs=[['ie_func1', ie_func1, [int], [int]], ['ie_func2', ie_func2, [str, int], [str]]]
)
     

'?sample_rule3(S,A,G)'

,S,A,G
0,Alice,25,Junior-Computer Science
4,Alice,25,Junior-English
8,Bob,22,Junior-Mathematics
9,Charlie,28,Senior-English
